<a href="https://colab.research.google.com/github/iesous-kurios/powerBi/blob/master/PowerBI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'0.10.1'

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
#@title
# import weekly data from yearly FPS folder

df = pd.read_excel('/gdrive/My Drive/FPData/YTD Data/updated_June18.xlsx', sheet_name=0)
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [4]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [5]:
currently_enrolled['3.3 Birthdate'].value_counts()

2009-01-01    2
1978-01-01    2
1980-01-01    2
1987-06-06    1
2018-03-25    1
             ..
2009-10-20    1
2013-09-11    1
2004-01-01    1
1962-09-19    1
2018-12-21    1
Name: 3.3 Birthdate, Length: 93, dtype: int64

In [6]:
bdays_date = (date.today()-timedelta(days=15)).isoformat()
current_bdays = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] >= bdays_date ]

In [7]:
from datetime import date

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [8]:
# Python3 code to  calculate age in years 
from datetime import date 
  
def turning(born): 
    today = date.today() 
    this_month = date.today().month
    this_day = date.today()
    try:  
        birthday = born.replace(year = today.year) 
        birthmonth = born.replace(month = today.month)
        birth_day = born.replace(day = today.day)
  
    # raised when birth date is February 29 
    # and the current year is not a leap year 
    except ValueError:  
        birthday = born.replace(year = today.year, 
                  month = this_month + 1, day = 1) 
  
    if birthday > today:
        birthmonth = born.month
        if birthmonth == this_month:
            if birth_day > this_day:
                return today.year - born.year + 1
            if birth_day <= this_day:
                return today.year - born.year 
        else:
            return today.year - born.year        

    else: 
        return today.year - born.year 
          
# Driver code 
print(turning(date(1997, 2, 3)), "years") 

23 years


In [9]:
current_bdays.head()

,2.1 Organization Name,2.2 Project Name,2.4 ProjectType,2.5 Utilization Tracking Method (Invalid),2.6 Federal Grant Programs,Enrollment Created By,3.1 FirstName,3.1 LastName,5.8 Personal ID,5.9 Household ID,3.15 Relationship to HoH,3.16 Client Location,CaseMembers,3.10 Enroll Date,3.11 Exit Date,3.12 Exit Destination,3.2 SocSecNo,3.2 Social Security Quality,3.3 Birthdate,3.3 Birthdate Quality,Age at Enrollment,Current Age,3.4 Race,3.5 Ethnicity,3.6 Gender,3.7 Veteran Status,3.8 Disabling Condition at Entry,3.917 Living Situation,3.917b Stayed Less Than 7 Nights,3.917 Stayed Less Than 90 Days,"3.917b Stayed in Streets, ES or SH Night Before",3.917 Length of Stay,3.917 Homeless Start Date,Length of Time Homeless (3.917 Approximate Start),3.917 Times Homeless Last 3 Years,3.917 Total Months Homeless Last 3 Years,V5 Last Permanent Address,V5 Prior Address,V5 State,V5 Zip,...,CaseAdults,Bed Nights During Report Period,Count of Bed Nights - Entire Episode,HEN-HP Referral Most Recent,HEN-RRH Referral Most Recent,WorkSource Referral Most Recent,YAHP Referral Most Recent,4.04.10 Other Public,4.04.10 State Funded,4.04.11 Indian Health Services (IHS),4.04.12 Other,4.04.3 Combined Childrens HealthInsurance/Medicaid,4.04.3 Medicaid,4.04.4 Medicare,4.04.5 State Children's health Insurance S-CHIP,4.04.6 Veteran's Administration Medical Services,4.04.8 Health Insurance obtained through COBRA,4.04.7 Private - Employer,4.04.9 Private,4.04.9 Private - Individual,4.2.3a Earned Income,4.2.4b Unemployment Insurance,4.2.5c Supplemental Security Income,4.2.6d Social Security Disability Income,4.2.7e VA Disability Compensation,4.2.8f VA Disability Pension,4.2.9g Private Disability Income,4.2.10h Workers Compensation,4.2.11i TANF,4.2.12j General Assistance,4.2.13k Retirement (Social Security),4.2.14l Pension from a Former Job,4.2.15m Child Support,4.2.16n Alimony,4.2.17o Other Income,Chronic Homeless Status_vHMISDatAssessment,Chronic Homeless Status_EvaluatevHMIS&HMISDA,Email,HomePhone,WorkPhone
254,Family Promise of Spokane,FPS--ES--Open Doors,Emergency shelter,Night-by-Night (nbn),NaN,Kermit Nelson,Anonymous,Anonymous76618,76618,122926,Self,Spokane City & County CoC,3,2019-10-26,NaT,NaN,NaN,Data not collected,1978-01-01,Approximate or Partial DOB Reported,41,42,White,Non-Hispanic/Latino,Male,No,Yes,Hotel or motel paid for without emergency shel...,NaN,NaN,NaN,One year or longer,2018-10-12,615.0,Two times,More than 12 months,Full address reported,3143 E 28th Ave,WA,99223.0,...,2,151.0,151.0,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,Yes,Yes,No,1800.0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,1,1,NaN,NaN,NaN
255,Family Promise of Spokane,FPS--ES--Open Doors,Emergency shelter,Night-by-Night (nbn),NaN,Kermit Nelson,Anonymous,Anonymous76619,76619,122926,Spouse,NaN,3,2019-10-26,NaT,NaN,NaN,Data not collected,1978-01-01,Approximate or Partial DOB Reported,41,42,White,Non-Hispanic/Latino,Female,No,Yes,Hotel or motel paid for without emergency shel...,NaN,NaN,NaN,One year or longer,2018-10-12,615.0,Two times,More than 12 months,NaN,NaN,NaN,NaN,...,2,151.0,151.0,NaN,NaN,NaN,NaN,No,Yes,No,No,No,No,No,No,No,No,No,No,No,0.0,0,0.0,0,0,0,0,0,0,0,0,0,200.0,0,0,1,1,NaN,NaN,NaN
256,Family Promise of Spokane,FPS--ES--Open Doors,Emergency shelter,Night-by-Night (nbn),NaN,Kermit Nelson,Anonymous,Anonymous76620,76620,122926,Daughter,NaN,3,2019-10-26,NaT,NaN,NaN,Data not collected,2016-01-01,Approximate or Partial DOB Reported,3,4,White,Non-Hispanic/Latino,Female,NaN,No,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,151.0,151.0,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,Yes,Yes,No,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,NaN,NaN,NaN
299,Family Promise of Spokane,FPS--ES--Open Doors,Emergency shelter,Night-by-Night (nbn),NaN,Kermit Nelson,Anden,Isak,97731,124767,Spouse,NaN,3,2019-11-23,NaT,NaN,586-13-0886,Full SSN,1977-02-11,Full DOB Reported,42,43,Native Hawaiian or Other Pacific Islander,Non-Hispanic/Latino,Male,No,No,Place not meant for habitation,NaN,NaN,NaN,One n

In [75]:
'''current_bdays['age'] = current_bdays['3.3 Birthdate'].apply(calculate_age)
current_bdays['age2'] = current_bdays['3.3 Birthdate'].apply(calculate_age)'''
current_bdays['turning'] = current_bdays['3.3 Birthdate'].apply(turning)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
bday_df = current_bdays[['3.1 FirstName', '3.1 LastName', '3.3 Birthdate','turning']].copy()

In [12]:

currentMonth = date.today().month
currentMonth
two_months = date.today().month + 2

last_month = date.today().month - 1

In [13]:
bday_df['month'] = pd.DatetimeIndex(bday_df['3.3 Birthdate']).month
bday_df['day'] = pd.DatetimeIndex(bday_df['3.3 Birthdate']).day

In [14]:
upcomming_bdays = bday_df[(bday_df['month'] > last_month) & (bday_df['month'] < two_months) ]

In [15]:
upcomming_bdays.to_excel('/gdrive/My Drive/FPData.close_bdays.xlsx')

In [16]:
upcomming_bdays

,3.1 FirstName,3.1 LastName,3.3 Birthdate,turning,month,day
301,T Merson,Doun,2014-06-06,6,6,6
404,Eric,Rideout,2003-06-01,17,6,1
503,Abigail Lea Fe,Striped Wolf,2015-06-01,5,6,1
505,Elizabeth,Green,1987-06-06,33,6,6
537,Deniko,Quintana,1997-06-04,23,6,4


In [17]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more



current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=30)).isoformat()

need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < days_before ]

In [18]:
exit_df = need_to_exit[['3.1 FirstName', '3.1 LastName', '5.8 Personal ID', '5.9 Household ID', '3.15 Relationship to HoH', 'CaseMembers', '3.10 Enroll Date', 'Date of Last ES Stay (Beta)']].copy()

In [19]:
#exit_df.head()

In [20]:
exit_df.to_excel('/gdrive/My Drive/FPData/YTD Data/YTDexit_df.xlsx')

In [21]:
total_served = df['5.8 Personal ID']
len(total_served)

544

In [22]:
households = df['5.9 Household ID'].unique()
len(households)

176

In [23]:
# print total number of families served (unduplicated)
unique_households = df['5.9 Household ID'].nunique()
unique_households

176

In [24]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [25]:
avg_bn = df['Count of Bed Nights (Housing Check-ins)'].mean()
avg_bn = round(avg_bn, 2)
avg_bn

43.11

In [26]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             119
Rental by client, no ongoing housing subsidy                                                                                      69
Rental by client, other ongoing housing subsidy                                                                                   53
Client refused                                                                                                                    37
Rental by client with RRH or equivalent subsidy                                                                                   25
Transitional Housing for homeless persons (including homeless youth)                                                              23
Staying or living with family, permanent tenure                                                                                   21
Place not meant for habitation (e.g., a vehicle, an abandoned buildin

In [27]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy',
                'Owned by client, no ongoing housing subsidy',
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [28]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [29]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [30]:
#df.head()

In [31]:
perm_leaver = df[df['perm_leaver'] == True]

In [32]:
total_perm = perm_leaver.perm_leaver.value_counts().sum()

In [33]:
total_perm

189

In [34]:
non_perm = df[df['perm_leaver'] == False]

In [35]:
total_non_perm = non_perm.perm_leaver.value_counts().sum()

In [36]:
total_non_perm

259

In [37]:
perm_percent = total_perm / (total_non_perm + total_perm)

In [38]:
perm_percent = round(perm_percent, 2)*100
perm_percent

42.0

In [39]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             119
Rental by client, no ongoing housing subsidy                                                                                      69
Rental by client, other ongoing housing subsidy                                                                                   53
Client refused                                                                                                                    37
Rental by client with RRH or equivalent subsidy                                                                                   25
Transitional Housing for homeless persons (including homeless youth)                                                              23
Staying or living with family, permanent tenure                                                                                   21
Place not meant for habitation (e.g., a vehicle, an abandoned buildin

In [40]:
unknowns = ['Client refused', 'No exit interview completed']

In [41]:
unknown_df = df[df['3.12 Exit Destination'].isin(unknowns)]

In [42]:
unknown_exits = unknown_df['3.12 Exit Destination'].value_counts().sum()

In [43]:
unknown_exits

57

In [44]:
unknown_percent = unknown_exits / total_non_perm
unknown_percent = round(unknown_percent, 2) *100
unknown_percent

22.0

In [45]:
exits = df['3.12 Exit Destination'].value_counts().sum()
exits

448

In [46]:
exits = df['3.12 Exit Destination'].value_counts()

In [47]:
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] 
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))
esExits
#print('Exits to Transitional',transExits,'%')

119

In [48]:
tempExits = df['3.12 Exit Destination'].isin(temp_reasons).sum()

tempExits

29

In [49]:
transitional = df['3.12 Exit Destination'] == 'Transitional Housing for homeless persons (including homeless youth)'
transitional = transitional.sum()
transitional

23

In [50]:
exits.sum()

448

In [51]:
total_guests = total_non_perm + total_perm

In [52]:
trans_percent = (transitional / exits.sum())*100
trans_percent = round(trans_percent, 2)
trans_percent

5.13

In [53]:
#zips_master

In [54]:
# export zips into an excel sheet for Power BI visual
zips_master = pd.read_excel('/gdrive/My Drive/FPData/uszips.xlsx')



# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
zips_bi = df['V5 Zip'].value_counts().to_frame('Total').reset_index(level=0, inplace=False)
zips_bi = zips_bi.rename(columns={"index": "V5 Zip"})
zips_bi = zips_bi.merge(zips_master,on='V5 Zip')
zips_bi['V5 City'].value_counts()

Spokane           10
Spokane Valley     5
Mountain Home      2
Greenacres         1
Colville           1
Bakersfield        1
Kennewick          1
Phoenix            1
Everett            1
Pullman            1
Cheney             1
Tacoma             1
Tillamook          1
Salt Lake City     1
Great Falls        1
Hemet              1
Wellpinit          1
Browning           1
Santa Rosa         1
Deer Park          1
Jensen Beach       1
Airway Heights     1
Hoodsport          1
Chewelah           1
Vancouver          1
Medical Lake       1
Lolo               1
Sprague            1
South Bend         1
Omak               1
Ogden              1
Nampa              1
Anderson           1
Name: V5 City, dtype: int64

In [55]:
type(zips_bi)

pandas.core.frame.DataFrame

In [56]:
zips_bi.to_excel('/gdrive/My Drive/FPData/YTD Data/weeklyYTDzips.xlsx', header=True, index=False)

In [57]:
other_reasons = ['Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)',
                 'Hotel or Motel paid for without Emergency Shelter Voucher', 'Other', 'Substance Abuse Treatment or Detox Center',
                 'Foster Care Home or Foster Care Group Home', 'Psychiatric Hospital or Other Psychiatric Facility']

In [58]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             119
Rental by client, no ongoing housing subsidy                                                                                      69
Rental by client, other ongoing housing subsidy                                                                                   53
Client refused                                                                                                                    37
Rental by client with RRH or equivalent subsidy                                                                                   25
Transitional Housing for homeless persons (including homeless youth)                                                              23
Staying or living with family, permanent tenure                                                                                   21
Place not meant for habitation (e.g., a vehicle, an abandoned buildin

In [59]:
other = df['3.12 Exit Destination'].isin(other_reasons)

In [60]:
other.sum()
other = other.sum()
other

31

In [61]:
df['4.04.7 Private - Employer']

0      No
1      No
2      No
3      No
4      No
       ..
506    No
521    No
534    No
535    No
536    No
Name: 4.04.7 Private - Employer, Length: 448, dtype: object

In [62]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             119
Rental by client, no ongoing housing subsidy                                                                                      69
Rental by client, other ongoing housing subsidy                                                                                   53
Client refused                                                                                                                    37
Rental by client with RRH or equivalent subsidy                                                                                   25
Transitional Housing for homeless persons (including homeless youth)                                                              23
Staying or living with family, permanent tenure                                                                                   21
Place not meant for habitation (e.g., a vehicle, an abandoned buildin

In [63]:
print(total_guests, 'total guests served YTD')
print(total_non_perm, 'total non perm leavers')
print(total_perm, 'total perm leavers')
print(esExits, 'total ES leavers')
print(tempExits, 'total temp leavers')
print(transitional, 'total trans leavers')
print(other, 'total leavers to "other"')
print(unknown_exits, 'total unknown leavers')
print('sum of all groups to make sure it matches total served:',total_perm+esExits+tempExits+transitional+other+unknown_exits)

448 total guests served YTD
259 total non perm leavers
189 total perm leavers
119 total ES leavers
29 total temp leavers
23 total trans leavers
31 total leavers to "other"
57 total unknown leavers
sum of all groups to make sure it matches total served: 448


In [64]:
total_perm+esExits+tempExits+transitional+other+unknown_exits

448

In [65]:
total_perm+total_non_perm

448

In [76]:
total_perm / (total_perm + total_non_perm)

0.421875

In [66]:
assert((total_guests) == (total_perm+esExits+tempExits+transitional+other+unknown_exits))

In [67]:
'''perm_bi = '{} %'.format(perm_percent)
es_bi = '{} %'.format(esExits)
temp_bi = '{} %'.format(tempExits)
trans_bi = '{} %'.format(trans_percent)
unknown_bi = '{} %'.format(unknown_percent)'''

"perm_bi = '{} %'.format(perm_percent)\nes_bi = '{} %'.format(esExits)\ntemp_bi = '{} %'.format(tempExits)\ntrans_bi = '{} %'.format(trans_percent)\nunknown_bi = '{} %'.format(unknown_percent)"

In [68]:
'''power_bi = pd.DataFrame({'Permanent Leavers': total_perm, 'Emergency Shelter Leavers': esExits, 
                         'Temporary Leavers': tempExits, 'Transitional Leavers': transitional, 
                         'Unknown Leavers': unknown_exits, 'Other Leavers': other, 'Total': total_guests }, index=[0])'''


"power_bi = pd.DataFrame({'Permanent Leavers': total_perm, 'Emergency Shelter Leavers': esExits, \n                         'Temporary Leavers': tempExits, 'Transitional Leavers': transitional, \n                         'Unknown Leavers': unknown_exits, 'Other Leavers': other, 'Total': total_guests }, index=[0])"

In [69]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data = [['Permanent Leavers', total_perm], ['Emergency Shelter Leavers', esExits], 
        ['Temporary Leavers', tempExits], ['Transitional Leavers', transitional],
        ['Unknown Leavers', unknown_exits], ['Other Leavers', other]] 
  


# Create the pandas DataFrame 
power_bi = pd.DataFrame(data, columns = ['Exit Destination', 'Total']) 
  
# print dataframe. 

In [70]:
power_bi

,Exit Destination,Total
0,Permanent Leavers,189
1,Emergency Shelter Leavers,119
2,Temporary Leavers,29
3,Transitional Leavers,23
4,Unknown Leavers,57
5,Other Leavers,31


In [71]:
power_bi.Total.sum()

448

In [72]:
power_bi.to_excel('/gdrive/My Drive/FPData/YTD Data/YTDweeklyDataNew.xlsx')

In [73]:
df['3.12 Exit Destination'].value_counts().sum()

448

In [74]:
assert(power_bi.Total.sum() == df['3.12 Exit Destination'].value_counts().sum())